# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

In [ ]:
import json
from bs4 import BeautifulSoup
with open("addres-book.json", "r") as file:
    data = json.load(file)
data

1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

In [ ]:
[i["email"] for i in data]

2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

In [ ]:
[[j["phone"] for j in i["phones"]] for i in data]

3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

In [ ]:
ad_book = BeautifulSoup(open("addres-book-q.xml","r"))
spisok = []
for i in ad_book.find_all("address"):
  name = i.find("name").get_text()
  for j in i.find_all("phone"):
    phone = j.get_text()
    spisok.append({name: phone})
spisok

## Лабораторная работа №4

### JSON

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [ ]:
with open('contributors_sample.json') as json_file:
    data = json.load(json_file)
for i in data[:3]:
    print(i)


1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [ ]:
set(line["mail"].split("@")[1] for line in data)

1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [ ]:
def person_name(username: str):
    for line in data:
        if username in line["username"]:
            return line
    raise ValueError

name = input(":")
try:
    print(person_name(name))
except ValueError:
    print("Пользователь не найден")

1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [ ]:
male_count = 0
female_count = 0

with open('contributors_sample.json') as json_file:
    data = json.load(json_file)
    
    for contributor in data:
        gender = contributor.get("sex")
        if gender == "M":
            male_count += 1
        elif gender == "F":
            female_count += 1
            
print("Number of males:", male_count)
print("Number of females:", female_count)

1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [ ]:
 contributors = pd.DataFrame(data, columns=['id', 'username', 'sex'])

1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [ ]:
import pandas as pd

recipes = pd.read_csv('recipes_sample.csv')
with open('contributors_sample.json') as json_file:
    data = json.load(json_file)
    contributors = pd.DataFrame(data, columns=['id', 'username', 'gender'])
merged_df = pd.merge(recipes, contributors, how='left', left_on='submitter_id', right_on='id')
merged_df.drop(columns=['id'], inplace=True)
missing_info_count = merged_df['username'].isnull().sum()
print(missing_info_count)

### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [ ]:
with open('contributors_sample.json') as json_file:
    data = json.load(json_file)

positions = {}

for item in data:
    position = item['должность']
    username = item['username']
    if position not in positions:
        positions[position] = []
    positions[position].append(username)

print(positions)

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [ ]:
import pickle

with open('job_people.pickle', 'wb') as pickle_file:
    pickle.dump(positions, pickle_file)

with open('job_people.json', 'w') as json_file:
    json.dump(positions, json_file, indent=4)

2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [ ]:
with open('job_people.pickle', 'rb') as pickle_file:
    positions = pickle.load(pickle_file)

for position, people in positions.items():
    print(position + ":")
    for person in people:
        print("  " + person)

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [ ]:
import xml.etree.ElementTree as ET


tree = ET.parse('steps_sample.xml')
root = tree.getroot()

steps_dict = {}

for recipe in root.findall('recipe'):
    recipe_id = recipe.get('id')
    steps = [step.text for step in recipe.findall('step')]
    steps_dict[recipe_id] = steps

with open('steps_sample.json', 'w') as json_file:
    json.dump(steps_dict, json_file)

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [ ]:
steps_count_dict = {}

for recipe in root.findall('recipe'):
    recipe_id = recipe.get('id')
    steps_count = len(recipe.findall('step'))
    if steps_count in steps_count_dict:
        steps_count_dict[steps_count].append(recipe_id)
    else:
        steps_count_dict[steps_count] = [recipe_id]

for count, recipe_ids in steps_count_dict.items():
    print(str(count) + " шагов:")
    for recipe_id in recipe_ids:
        print("  " + recipe_id)

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [ ]:
time_recipes = []

for recipe in root.findall('recipe'):
    for step in recipe.findall('step'):
        for instruction in step.findall('instruction'):
            if 'час' in instruction.text.lower() or 'минут' in instruction.text.lower():
                time_recipes.append(recipe.get('id'))
                break
        else:
            continue
        break

print(time_recipes)

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [ ]:
recipes = pd.read_csv('recipes_sample.csv')
for i, row in recipes.iterrows():
    recipe_id = row['id']
    n_steps = row['n_steps']
    if pd.isna(n_steps):
        if recipe_id in steps_dict:
            recipes.at[i, 'n_steps'] = len(steps_dict[recipe_id])

# Сохранение измененной таблицы recipes в файл recipes_updated.csv
recipes.to_csv('recipes_updated.csv', index=False)

3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [ ]:
if recipes['n_steps'].isna().sum() == 0:
    # преобразование столбца к целочисленному типу
    recipes['n_steps'] = recipes['n_steps'].astype(int)
    # сохранение результатов
    recipes.to_csv('recipes_sample_with_filled_nsteps.csv', index=False)
    print('Столбец n_steps не содержит пропусков и успешно преобразован к целочисленному типу.')
else:
    print('Столбец n_steps содержит пропуски.')